# Сопоставление дескрипторов

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os

In [ ]:
# Загружаем два изображения для сопоставления
img1_path = 'sample1.jpg'
img2_path = 'sample2.jpg'

def load_or_create_image_pair():
    """Загружает или создаёт пару изображений для сопоставления"""

    # Пытаемся загрузить готовые изображения
    if os.path.exists(img1_path) and os.path.exists(img2_path):
        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)
        print(f"✅ Изображения загружены: {img1_path}, {img2_path}")
        return img1, img2

    print("📝 Создаём синтетическую пару изображений...")

    # Создаём базовое изображение
    img1 = np.ones((400, 600, 3), dtype=np.uint8) * 230

    # Добавляем узнаваемые объекты
    # Прямоугольники
    cv2.rectangle(img1, (50, 50), (150, 150), (100, 150, 200), -1)
    cv2.rectangle(img1, (200, 100), (300, 200), (200, 100, 100), -1)

    # Круги
    cv2.circle(img1, (450, 100), 40, (150, 200, 150), -1)
    cv2.circle(img1, (450, 100), 25, (100, 150, 100), 3)

    # Треугольник
    triangle = np.array([[100, 250], [200, 250], [150, 320]], np.int32)
    cv2.fillPoly(img1, [triangle], (180, 180, 100))

    # Текст
    cv2.putText(img1, 'MATCH', (350, 300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (80, 80, 80), 3)

    # Шахматная доска для углов
    square_size = 25
    for i in range(6):
        for j in range(6):
            if (i + j) % 2 == 0:
                y1 = 250 + i * square_size
                x1 = 350 + j * square_size
                cv2.rectangle(img1, (x1, y1), (x1+square_size, y1+square_size), (60, 60, 60), -1)

    # Создаём второе изображение с трансформацией
    height, width = img1.shape[:2]
    center = (width // 2, height // 2)

    # Поворот на 30° + масштаб 1.2
    M = cv2.getRotationMatrix2D(center, 30, 1.2)
    img2 = cv2.warpAffine(img1, M, (width, height))

    # Добавляем изменение яркости
    img2 = np.clip(img2.astype(np.int16) + 30, 0, 255).astype(np.uint8)

    # Добавляем шум
    noise = np.random.randint(-15, 15, img2.shape, dtype=np.int16)
    img2 = np.clip(img2.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    print("✅ Синтетические изображения созданы")
    return img1, img2

img1, img2 = load_or_create_image_pair()

# Преобразуем в grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Визуализируем исходные изображения
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
axes[0].set_title('Изображение 1 (исходное)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
axes[1].set_title('Изображение 2 (трансформированное)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Детекция ключевых точек на обоих изображениях
# ===============================

# Инициализируем детекторы
print("🔧 Инициализация детекторов...")

detectors = {}

# SIFT
try:
    sift = cv2.SIFT_create()
    kp1_sift, des1_sift = sift.detectAndCompute(img1_gray, None)
    kp2_sift, des2_sift = sift.detectAndCompute(img2_gray, None)
    detectors['SIFT'] = {
        'detector': sift,
        'kp1': kp1_sift, 'des1': des1_sift,
        'kp2': kp2_sift, 'des2': des2_sift,
        'norm': cv2.NORM_L2
    }
    print(f"✅ SIFT: {len(kp1_sift)} точек (img1), {len(kp2_sift)} точек (img2)")
except:
    print("❌ SIFT недоступен")

# ORB
try:
    orb = cv2.ORB_create(nfeatures=500)
    kp1_orb, des1_orb = orb.detectAndCompute(img1_gray, None)
    kp2_orb, des2_orb = orb.detectAndCompute(img2_gray, None)
    detectors['ORB'] = {
        'detector': orb,
        'kp1': kp1_orb, 'des1': des1_orb,
        'kp2': kp2_orb, 'des2': des2_orb,
        'norm': cv2.NORM_HAMMING
    }
    print(f"✅ ORB: {len(kp1_orb)} точек (img1), {len(kp2_orb)} точек (img2)")
except:
    print("❌ ORB недоступен")

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Базовое сопоставление с BFMatcher
# ===============================

matching_results = {}

for detector_name, detector_data in detectors.items():
    print(f"\n🔍 Сопоставление для {detector_name}:")

    # Создаём BFMatcher
    bf = cv2.BFMatcher(detector_data['norm'], crossCheck=True)

    # Сопоставляем дескрипторы
    start_time = time.time()
    matches = bf.match(detector_data['des1'], detector_data['des2'])
    matching_time = time.time() - start_time

    # Сортируем по расстоянию (лучшие первыми)
    matches = sorted(matches, key=lambda x: x.distance)

    print(f"   Найдено соответствий: {len(matches)}")
    print(f"   Время сопоставления: {matching_time:.4f} сек")

    # Анализ расстояний
    distances = [m.distance for m in matches]
    print(f"   Расстояния:")
    print(f"      Мин: {min(distances):.2f}")
    print(f"      Макс: {max(distances):.2f}")
    print(f"      Среднее: {np.mean(distances):.2f}")
    print(f"      Медиана: {np.median(distances):.2f}")

    matching_results[detector_name] = {
        'matches': matches,
        'time': matching_time,
        'distances': distances
    }

# Визуализируем соответствия
if matching_results:
    num_detectors = len(matching_results)
    fig, axes = plt.subplots(num_detectors, 2, figsize=(16, 6*num_detectors))

    if num_detectors == 1:
        axes = axes.reshape(1, -1)

    for i, (detector_name, result) in enumerate(matching_results.items()):
        detector_data = detectors[detector_name]

        # Все соответствия
        img_matches_all = cv2.drawMatches(
            img1_gray, detector_data['kp1'],
            img2_gray, detector_data['kp2'],
            result['matches'], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        axes[i, 0].imshow(img_matches_all)
        axes[i, 0].set_title(f'{detector_name}: Все соответствия ({len(result["matches"])})')
        axes[i, 0].axis('off')

        # Топ-20 лучших соответствий
        img_matches_top = cv2.drawMatches(
            img1_gray, detector_data['kp1'],
            img2_gray, detector_data['kp2'],
            result['matches'][:20], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        axes[i, 1].imshow(img_matches_top)
        axes[i, 1].set_title(f'{detector_name}: Топ-20 лучших соответствий')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Гистограммы расстояний
if matching_results:
    fig, axes = plt.subplots(1, len(matching_results), figsize=(7*len(matching_results), 5))

    if len(matching_results) == 1:
        axes = [axes]

    for i, (detector_name, result) in enumerate(matching_results.items()):
        axes[i].hist(result['distances'], bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_xlabel('Расстояние')
        axes[i].set_ylabel('Количество соответствий')
        axes[i].set_title(f'{detector_name}: Распределение расстояний')
        axes[i].axvline(np.median(result['distances']), color='red',
                       linestyle='--', label=f'Медиана: {np.median(result["distances"]):.2f}')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Тест отношения Лоу (Lowe's Ratio Test)
# ===============================

def lowe_ratio_test(matches_knn, ratio=0.75):
    """
    Применяет тест отношения Лоу для фильтрации соответствий

    Args:
        matches_knn: список k ближайших соседей (k=2)
        ratio: порог отношения (обычно 0.7-0.8)

    Returns:
        good_matches: отфильтрованные соответствия
    """
    good_matches = []

    for match_pair in matches_knn:
        if len(match_pair) == 2:  # Убеждаемся что есть 2 соседа
            m, n = match_pair

            # Тест отношения
            if m.distance < ratio * n.distance:
                good_matches.append(m)

    return good_matches

lowe_results = {}

for detector_name, detector_data in detectors.items():
    print(f"\n🔍 Тест Лоу для {detector_name}:")

    # Создаём BFMatcher без crossCheck для knnMatch
    bf = cv2.BFMatcher(detector_data['norm'], crossCheck=False)

    # Находим 2 ближайших соседа для каждого дескриптора
    matches_knn = bf.knnMatch(detector_data['des1'], detector_data['des2'], k=2)

    # Тестируем разные значения ratio
    ratios = [0.6, 0.7, 0.75, 0.8, 0.9]
    ratio_results = {}

    for ratio in ratios:
        good_matches = lowe_ratio_test(matches_knn, ratio)
        ratio_results[ratio] = good_matches
        print(f"   Ratio {ratio}: {len(good_matches)} соответствий "
              f"({len(good_matches)/len(matches_knn)*100:.1f}% от исходных)")

    lowe_results[detector_name] = ratio_results

# Визуализируем влияние параметра ratio
if lowe_results:
    for detector_name, ratio_results in lowe_results.items():
        detector_data = detectors[detector_name]

        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()

        # Исходные соответствия (без фильтрации)
        bf = cv2.BFMatcher(detector_data['norm'], crossCheck=True)
        matches_original = bf.match(detector_data['des1'], detector_data['des2'])

        img_original = cv2.drawMatches(
            img1_gray, detector_data['kp1'],
            img2_gray, detector_data['kp2'],
            matches_original[:50], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        axes[0].imshow(img_original)
        axes[0].set_title(f'{detector_name}: Исходные\n({len(matches_original)} соответствий)')
        axes[0].axis('off')

        # Результаты с разными ratio
        for i, (ratio, good_matches) in enumerate(ratio_results.items(), 1):
            if i < len(axes):
                img_filtered = cv2.drawMatches(
                    img1_gray, detector_data['kp1'],
                    img2_gray, detector_data['kp2'],
                    good_matches[:50], None,
                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
                )

                axes[i].imshow(img_filtered)
                axes[i].set_title(f'Ratio = {ratio}\n({len(good_matches)} соответствий)')
                axes[i].axis('off')

        plt.suptitle(f'{detector_name}: Влияние теста отношения Лоу', fontsize=14)
        plt.tight_layout()
        plt.show()

# График зависимости количества соответствий от ratio
if lowe_results:
    fig, ax = plt.subplots(figsize=(10, 6))

    for detector_name, ratio_results in lowe_results.items():
        ratios = list(ratio_results.keys())
        counts = [len(ratio_results[r]) for r in ratios]

        ax.plot(ratios, counts, marker='o', linewidth=2, markersize=8, label=detector_name)

    ax.set_xlabel('Порог ratio')
    ax.set_ylabel('Количество соответствий')
    ax.set_title('Влияние параметра ratio на количество соответствий')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Сравнение BFMatcher vs FLANN
# ===============================

flann_results = {}

for detector_name, detector_data in detectors.items():
    print(f"\n⚡ Сравнение для {detector_name}:")

    # BFMatcher
    bf = cv2.BFMatcher(detector_data['norm'], crossCheck=False)

    start_time = time.time()
    bf_matches = bf.knnMatch(detector_data['des1'], detector_data['des2'], k=2)
    bf_time = time.time() - start_time

    bf_good = lowe_ratio_test(bf_matches, ratio=0.75)

    # FLANN
    if detector_name == 'SIFT':
        # Для SIFT используем KD-Tree
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
    else:  # ORB
        # Для ORB используем LSH
        FLANN_INDEX_LSH = 6
        index_params = dict(
            algorithm=FLANN_INDEX_LSH,
            table_number=6,
            key_size=12,
            multi_probe_level=1
        )
        search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    start_time = time.time()
    flann_matches = flann.knnMatch(detector_data['des1'], detector_data['des2'], k=2)
    flann_time = time.time() - start_time

    flann_good = lowe_ratio_test(flann_matches, ratio=0.75)

    print(f"   BFMatcher:")
    print(f"      Время: {bf_time:.4f} сек")
    print(f"      Хорошие соответствия: {len(bf_good)}")

    print(f"   FLANN:")
    print(f"      Время: {flann_time:.4f} сек")
    print(f"      Хорошие соответствия: {len(flann_good)}")
    print(f"      Ускорение: {bf_time/flann_time:.2f}x")

    flann_results[detector_name] = {
        'bf_time': bf_time,
        'bf_matches': bf_good,
        'flann_time': flann_time,
        'flann_matches': flann_good
    }

# Визуализация сравнения
if flann_results:
    fig, axes = plt.subplots(len(flann_results), 2, figsize=(16, 6*len(flann_results)))

    if len(flann_results) == 1:
        axes = axes.reshape(1, -1)

    for i, (detector_name, results) in enumerate(flann_results.items()):
        detector_data = detectors[detector_name]

        # BFMatcher результаты
        img_bf = cv2.drawMatches(
            img1_gray, detector_data['kp1'],
            img2_gray, detector_data['kp2'],
            results['bf_matches'][:30], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        axes[i, 0].imshow(img_bf)
        axes[i, 0].set_title(
            f'{detector_name} - BFMatcher\n'
            f'{len(results["bf_matches"])} соответствий, {results["bf_time"]:.4f} сек'
        )
        axes[i, 0].axis('off')

        # FLANN результаты
        img_flann = cv2.drawMatches(
            img1_gray, detector_data['kp1'],
            img2_gray, detector_data['kp2'],
            results['flann_matches'][:30], None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        axes[i, 1].imshow(img_flann)
        axes[i, 1].set_title(
            f'{detector_name} - FLANN\n'
            f'{len(results["flann_matches"])} соответствий, {results["flann_time"]:.4f} сек'
        )
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Система поиска похожих изображений
# ===============================

def create_image_database(n_images=10):
    """Создаёт базу данных изображений с различными трансформациями"""

    database = []

    # Базовое изображение
    base_img = img1.copy()
    base_gray = img1_gray.copy()

    print(f"🔄 Создаём базу из {n_images} изображений...")

    for i in range(n_images):
        # Применяем случайные трансформации
        angle = np.random.randint(-45, 45)
        scale = np.random.uniform(0.7, 1.3)
        brightness = np.random.randint(-30, 30)

        # Поворот и масштаб
        h, w = base_gray.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, scale)
        transformed = cv2.warpAffine(base_gray, M, (w, h))

        # Изменение яркости
        transformed = np.clip(transformed.astype(np.int16) + brightness, 0, 255).astype(np.uint8)

        # Добавляем шум
        noise = np.random.randint(-20, 20, transformed.shape, dtype=np.int16)
        transformed = np.clip(transformed.astype(np.int16) + noise, 0, 255).astype(np.uint8)

        database.append({
            'image': transformed,
            'id': i,
            'transform': f'angle={angle}°, scale={scale:.2f}, bright={brightness}'
        })

    print(f"✅ База данных создана: {len(database)} изображений")
    return database

def search_similar_images(query_img, database, detector, top_k=5):
    """Ищет похожие изображения в базе данных"""

    print("🔍 Поиск похожих изображений...")

    # Детектируем ключевые точки в запросе
    kp_query, des_query = detector.detectAndCompute(query_img, None)

    if des_query is None or len(des_query) == 0:
        print("❌ Не найдено ключевых точек в запросе")
        return []

    # Определяем норму для BFMatcher
    if isinstance(detector, cv2.SIFT):
        norm = cv2.NORM_L2
    else:
        norm = cv2.NORM_HAMMING

    bf = cv2.BFMatcher(norm, crossCheck=False)

    similarities = []

    for db_item in database:
        # Детектируем ключевые точки в изображении из базы
        kp_db, des_db = detector.detectAndCompute(db_item['image'], None)

        if des_db is None or len(des_db) == 0:
            similarities.append(0)
            continue

        # Сопоставляем
        try:
            matches = bf.knnMatch(des_query, des_db, k=2)
            good_matches = lowe_ratio_test(matches, ratio=0.75)

            # Количество хороших соответствий = мера похожести
            similarity = len(good_matches)
        except:
            similarity = 0

        similarities.append(similarity)

    # Ранжируем по убыванию похожести
    ranked_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in ranked_indices:
        results.append({
            'id': database[idx]['id'],
            'image': database[idx]['image'],
            'transform': database[idx]['transform'],
            'similarity': similarities[idx]
        })

    return results

# Создаём базу данных
image_database = create_image_database(n_images=12)

# Выбираем детектор для поиска
if 'SIFT' in detectors:
    search_detector = detectors['SIFT']['detector']
    detector_name_search = 'SIFT'
elif 'ORB' in detectors:
    search_detector = detectors['ORB']['detector']
    detector_name_search = 'ORB'
else:
    print("❌ Нет доступных детекторов для поиска")
    search_detector = None

# Выполняем поиск
if search_detector is not None:
    # Создаём запросное изображение (трансформация оригинала)
    query_img = img2_gray.copy()

    results = search_similar_images(query_img, image_database, search_detector, top_k=6)

    # Визуализируем результаты
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()

    # Запросное изображение
    axes[0].imshow(query_img, cmap='gray')
    axes[0].set_title('Запрос (query)')
    axes[0].axis('off')
    axes[0].set_facecolor('lightcoral')

    # Результаты поиска
    for i, result in enumerate(results, 1):
        if i < len(axes):
            axes[i].imshow(result['image'], cmap='gray')
            axes[i].set_title(
                f'#{i}: ID={result["id"]}\n'
                f'Соответствий: {result["similarity"]}\n'
                f'{result["transform"][:30]}...'
            )
            axes[i].axis('off')

    # Скрываем неиспользованные оси
    for i in range(len(results)+1, len(axes)):
        axes[i].axis('off')

    plt.suptitle(f'Результаты поиска похожих изображений ({detector_name_search})', fontsize=14)
    plt.tight_layout()
    plt.show()

    print("\n📊 Топ-5 результатов:")
    for i, result in enumerate(results[:5], 1):
        print(f"   {i}. ID {result['id']}: {result['similarity']} соответствий")

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте RANSAC для фильтрации выбросов:
- Случайно выбирайте 4 соответствия
- Вычисляйте гомографию
- Подсчитывайте inliers
- Повторяйте N раз, выбирайте лучшую модель

2. Оптимизируйте параметры FLANN:
- Для SIFT: trees (1-10), checks (10-100)
- Для ORB: table_number, key_size, multi_probe_level
- Найдите оптимальный баланс скорость/качество

3. Создайте мини-проект 'Поиск объекта':
- Загрузите изображение объекта (шаблон)
- Ищите этот объект на сложной сцене
- Нарисуйте ограничивающий прямоугольник вокруг найденного объекта